In [72]:
import matplotlib.pyplot as plt
import random
import numpy as np
from scipy.stats import beta
from matplotlib import pyplot as plt

In [73]:
class Batch:
  quality = 0.5
  alpha = 1
  beta = 1
  good = 0
  

  def __init__(self, quality):
    self.quality = quality
  

  def sample_distribution(self):
    return np.random.beta(self.alpha, self.beta)
  
  
  def get_element(self):
    good = np.random.rand() < self.quality
    self.alpha = self.alpha + int(good)
    self.beta = self.beta + 1 - int(good)
    return good

In [74]:
class SwapLemon:
  batches = []
  systems = 0
  good = 0
  current_batch = 0

  def set_batches(self, qualities):
    self.batches = [Batch(quality) for quality in qualities]
    self.systems = 0
    self.good = 0
  
  def get_element(self):
    if self.batches[self.current_batch].get_element():
        self.systems += 1
        self.good += 1
    else:
        self.systems += 1
        if self.current_batch < len(batches)-1:
            self.current_batch += 1
        else:
            self.current_batch = 0

In [75]:
class nBayes:
  batches = []
  systems = 0
  good = 0
  current_batch = 0
  epoch = 0
  training = 0
  in_training = True
  order =[]


  def set_batches(self, qualities, training):
    self.batches = [Batch(quality) for quality in qualities]
    self.systems = 0
    self.good = 0
    self.training = len(batches)*training
    self.order = []


  def get_element(self):
    if self.in_training:  
        self.epoch +=1

        self.systems += 1
        if self.batches[self.current_batch].get_element():
            self.good += 1
            self.batches[self.current_batch].good += 1
            
        if self.current_batch < len(batches)-1:
            self.current_batch += 1
        else:
            self.current_batch = 0

        if self.epoch >= self.training:
            self.in_training = False
            self.epoch = 0
            order = []
            scores = [x.good for x in self.batches] 
            for i in range(len(scores)):
                for _ in range(len(batches)-i):
                    self.order.append(np.argmax(scores))
                scores[np.argmax(scores)] = -1

    else:
        if self.epoch > len(self.order)-1:
            self.epoch = 0
        
        self.systems += 1
        if self.batches[self.order[self.epoch]].get_element():
            self.good += 1
        
        self.epoch += 1

In [76]:
class sBayes:
    batches = []
    systems = 0
    good = 0
    current_batch = 0
    epoch = 0
    training = 0
    in_training = True
    order =[]


    def set_batches(self, qualities, training):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0
        self.training = len(batches)*training
        self.order = []


    def get_element(self):
        if self.in_training:  
            self.epoch +=1
    
            self.systems += 1
            if self.batches[self.current_batch].get_element():
                self.good += 1
                self.batches[self.current_batch].good += 1
                
            if self.current_batch < len(batches)-1:
                self.current_batch += 1
            else:
                self.current_batch = 0
    
            if self.epoch >= self.training:
                self.in_training = False
                self.epoch = 0
                scores = [x.good for x in self.batches]
                scores2 = scores.copy()
    
                last_count = 1
                for i in range(len(scores)):
                    if i == 0:
                        self.order.append(np.argmin(scores))
                        scores[np.argmin(scores)] = 1_000_000
                    else:
                        si_minus_1 = scores2[self.order[-1]]
                        si = scores2[i]
                        if si_minus_1 == 0:
                            si_minus_1 = 1
                        last_count *= round(si/si_minus_1)
                        for a in range(last_count):
                            self.order.append(np.argmin(scores))
                        scores[np.argmin(scores)] = 1_000_000
                self.order.reverse()

        else:
            if self.epoch > len(self.order)-1:
                self.epoch = 0
            
            self.systems += 1
            if self.batches[self.order[self.epoch]].get_element():
                self.good += 1
            
            self.epoch += 1

In [77]:
class MyBayes:
    batches = []
    systems = 0
    good = 0
    current_batch = 0
    epoch = 0
    training = 0
    in_training = True
    order =[]
    
    
    def set_batches(self, qualities, training):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0
        self.training = len(batches)*training
        self.order = []
    

    def get_element(self):
        if self.in_training:  
            self.epoch +=1
            self.systems += 1
            if self.batches[self.current_batch].get_element():
                self.good += 1
                self.batches[self.current_batch].good += 1
                
            if self.current_batch < len(batches)-1:
                self.current_batch += 1
    
            else:
                self.current_batch = 0
    
            if self.epoch >= self.training:
              self.in_training = False
              self.epoch = 0
              order = []
              scores = [x.good for x in self.batches] 
              for i in range(len(scores)):
                  for a in range(len(batches)-i):
                      self.order.append(np.argmax(scores))
    
                  scores[np.argmax(scores)] = -1
    
              self.current_batch = self.order[0]
    
        else:
          self.systems+=1
          if self.batches[self.current_batch].get_element():
            self.good += 1
    
          else:
            self.current_batch = random.choice(self.order)

In [78]:
tries = 100 
epoch = 300 
learning = 30
batches = [0.98, 0.75, 0.85, 0.8, 0.6]
swap_lemon_score = 0
n_bayes_score = 0
s_bayes_score = 0
my_bayes_score = 0


for _ in range(tries):
    swap_lemon = SwapLemon()
    swap_lemon.set_batches(batches)
    for i in range(epoch):
        swap_lemon.get_element()
    
    swap_lemon_score += swap_lemon.good/swap_lemon.systems
    n_bayes = nBayes()
    n_bayes.set_batches(batches, learning)
    for i in range(epoch):
        n_bayes.get_element()
    
    n_bayes_score += n_bayes.good/n_bayes.systems
    s_bayes = sBayes()
    s_bayes.set_batches(batches, learning)
    for i in range(epoch):
        s_bayes.get_element()
    
    s_bayes_score += s_bayes.good/s_bayes.systems
    my_bayes = MyBayes()
    my_bayes.set_batches(batches, learning)
    for i in range(epoch):
        my_bayes.get_element()

    my_bayes_score += my_bayes.good/my_bayes.systems

print(f'Average succes ratio for swap on bad strategy: {swap_lemon_score/tries:.2%}')
print(f'Average succes ratio for rank n strategy: {n_bayes_score/tries:.2%}')
print(f'Average succes ratio for rank s strategy: {s_bayes_score/tries:.2%}')
print(f'Average succes ratio for my strategy: {my_bayes_score/tries:.2%}') 

Average succes ratio for swap on bad strategy: 93.24%
Average succes ratio for rank n strategy: 82.21%
Average succes ratio for rank s strategy: 79.85%
Average succes ratio for my strategy: 87.46%


In [79]:
tries = 100 
epoch = 300 
learning = 30
batches = [0.98, 0.75, 0.85, 0.8, 0.6]
swapFactoryScore = 0
rankNFactoryScore = 0
rankSFactoryScore = 0
BayesianFactoryScore = 0
myFactoryScore = 0
fbFactoryScore = 0

for _ in range(tries): 
  swapFactory = swap_on_bad_Factory()
  swapFactory.set_batches(batches)
  for i in range(epoch):
    swapFactory.get_element()
  swapFactoryScore += swapFactory.good/swapFactory.systems

  rankNFactory = rank_n_Factory()
  rankNFactory.set_batches(batches, learning)
  for i in range(epoch):
    rankNFactory.get_element()
  rankNFactoryScore += rankNFactory.good/rankNFactory.systems

  rankSFactory = rank_s_Factory()
  rankSFactory.set_batches(batches, learning)
  for i in range(epoch):
    rankSFactory.get_element()
  rankSFactoryScore += rankSFactory.good/rankSFactory.systems

  BayesianFactory = Bayesian_Factory()
  BayesianFactory.set_batches(batches)
  for i in range(epoch):
    BayesianFactory.get_element()
  BayesianFactoryScore += BayesianFactory.good/BayesianFactory.systems

  myFactory = my_Factory()
  myFactory.set_batches(batches, learning)
  for i in range(epoch):
    myFactory.get_element()
  myFactoryScore += myFactory.good/myFactory.systems

  fbFactory = Forgetting_Bayesian_Factory()
  fbFactory.set_batches(batches)
  for i in range(epoch):
    fbFactory.get_element()
  fbFactoryScore += fbFactory.good/fbFactory.systems

print(f'Average succes ratio for swap on bad strategy: {swapFactoryScore/tries:.2%}')
print(f'Average succes ratio for rank n strategy: {rankNFactoryScore/tries:.2%}')
print(f'Average succes ratio for rank s strategy: {rankSFactoryScore/tries:.2%}')
print(f'Average succes ratio for Bayesian strategy: {BayesianFactoryScore/tries:.2%}')
print(f'Average succes ratio for my strategy: {myFactoryScore/tries:.2%}')
print(f'Average succes ratio for forgetting Bayesian strategy: {fbFactoryScore/tries:.2%}')

Average succes ratio for swap on bad strategy: 92.78%
Average succes ratio for rank n strategy: 82.24%
Average succes ratio for rank s strategy: 79.79%
Average succes ratio for Bayesian strategy: 95.40%
Average succes ratio for my strategy: 87.35%
Average succes ratio for forgetting Bayesian strategy: 89.33%
